# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OLLAMA_HOST_URL=http://localhost:11434


In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [12]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [5]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [6]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [7]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
import os

model = ChatOllama(
    model="granite4:latest",
    temperature=0,
    base_url=os.environ['OLLAMA_HOST_URL']
)

agent = create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [13]:
question = "This is Frank Harris, What was the total on my last invoice? Maybe you can look at Invoice table"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice? Maybe you can look at Invoice table
================================== Ai Message ==================================
Tool Calls:
  execute_sql (c61c695f-deab-45f0-bd5b-4a2e3f115a74)
 Call ID: c61c695f-deab-45f0-bd5b-4a2e3f115a74
  Args:
    query: SELECT SUM(total) AS total FROM Invoice ORDER BY date DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such column: date
[SQL: SELECT SUM(total) AS total FROM Invoice ORDER BY date DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (66df0729-af90-448c-b8e4-d4aff3717bf3)
 Call ID: 66df0729-af90-448c-b8e4-d4aff3717bf3
  Args:
    query: SELECT SUM(total) AS total FROM Invoic

In [14]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

I’m sorry, but I don’t know which table or database you’re referring to when you ask about “the titles.” Could you please provide more details—such as the name of the table (or tables) that contain the title data, any relevant columns, and perhaps a brief description of what you need? With that information I can run an appropriate `SELECT` query for you.


## Add memory

In [15]:
from langgraph.checkpoint.memory import InMemorySaver

In [16]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
from langchain_ollama import ChatOllama
import os

model = ChatOllama(
    model="granite4:latest",
    temperature=0,
    base_url=os.environ['OLLAMA_HOST_URL']
)

agent = create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [17]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (251b9380-e55e-424e-af4c-efc283f25285)
 Call ID: 251b9380-e55e-424e-af4c-efc283f25285
  Args:
    query: SELECT SUM(total_amount) AS total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY date DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT SUM(total_amount) AS total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY date DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (b542e69a-553f-4640-8ad6-074acd0b7155)
 Call ID: b542e69a-553f-4640-8ad6-074acd0b7155


In [18]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (c65f0b05-5ce4-494b-82cd-9d58051dc618)
 Call ID: c65f0b05-5ce4-494b-82cd-9d58051dc618
  Args:
    query: SELECT title FROM movies WHERE director = 'Frank Harris';
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: movies
[SQL: SELECT title FROM movies WHERE director = 'Frank Harris';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================

I'm sorry, but the database query failed because it could not find a table named **movies**. This suggests that either the `movies` table does not exist in your SQLite database or there is another issue with the schema.

To resolve this, please verify:

1. **Tab

## Try your own queries
Now that there is memory, check the agents recall!

In [ ]:
question = "Your Question Here?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)